In [ ]:
import pandas as pd
import os
import json


In [ ]:


# Opening JSON file


In [12]:
root = '../../'
rel_path = r"Results/HPO/TripletLoss"

data_list = []

for folder in os.listdir(root+rel_path):
    folderPath = root+rel_path+"/"+folder
    

    
    for file in os.listdir(folderPath):
        filePath = f"{folderPath}/{file}"
        with open( filePath ) as json_file:
            data = json.load(json_file)
            data["filePath"] = filePath

            data_list.append( data )



In [ ]:
data_list